> **NOTE**: This tutorial will demonstrate how to utilize the GPU provided by Colab to run LLM with Xinference local server, and how to interact with the model in different ways (OpenAI-Compatible endpoints/Xinference's builtin Client/LangChain).


# Xinference

Xorbits Inference (Xinference) is an open-source platform to streamline the operation and integration of a wide array of AI models. With Xinference, you’re empowered to run inference using any open-source LLMs, embedding models, and multimodal models either in the cloud or on your own premises, and create robust AI-driven applications.




* [Docs](https://inference.readthedocs.io/en/latest/index.html)
* [Built-in Models](https://inference.readthedocs.io/en/latest/models/builtin/index.html)
* [Custom Models](https://inference.readthedocs.io/en/latest/models/custom.html)
* [Deployment Docs](https://inference.readthedocs.io/en/latest/getting_started/using_xinference.html)
* [Examples and Tutorials](https://inference.readthedocs.io/en/latest/examples/index.html)


## Set up the environment

> **NOTE**: We recommend you run this demo on a GPU. To change the runtime type: In the toolbar menu, click **Runtime** > **Change runtime typ**e > **Select the GPU (T4)**


### Check memory and GPU resources

In [1]:
import psutil
import torch


ram = psutil.virtual_memory()
ram_total = ram.total / (1024**3)
print('RAM: %.2f GB' % ram_total)

print('=============GPU INFO=============')
if torch.cuda.is_available():
  !/opt/bin/nvidia-smi || ture
else:
  print('GPU NOT available')

RAM: 12.67 GB
=============GPU INFO=============
Sat Dec 14 08:44:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+------------------

### Install Xinference and dependencies

In [9]:
%pip install -U -q xinference[transformers] openai langchain

In [3]:
!pip show xinference

Name: xinference
Version: 1.1.0
Summary: Model Serving Made Easy
Home-page: https://github.com/xorbitsai/inference
Author: Qin Xuye
Author-email: qinxuye@xprobe.io
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aioprometheus, async-timeout, click, fastapi, gradio, huggingface-hub, modelscope, nvidia-ml-py, openai, passlib, peft, pillow, pydantic, python-jose, requests, setproctitle, sse-starlette, tabulate, timm, torch, tqdm, typing-extensions, uvicorn, xoscar
Required-by: 


## A Quick Start Demo
### Start Local Server


To start a local instance of Xinference, run `xinference` in the background via `nohup`:

In [14]:
!nohup xinference-local  > xinference.log 2>&1 &

Congrats! You now have Xinference running in Colab machine. The default host and ip is 127.0.0.1 and 9997 respectively.


Once Xinference is running, there are multiple ways we can try it: via the web UI, via cURL, via the command line, or via the Xinference’s python client.

The command line tool is `xinference`. You can list the commands that can be used by running:

In [5]:
!xinference --help

Usage: xinference [OPTIONS] COMMAND [ARGS]...

  Xinference command-line interface for serving and deploying models.

Options:
  -v, --version       Show the current version of the Xinference tool.
  --log-level TEXT    Set the logger level. Options listed from most log to
                      least log are: DEBUG > INFO > WARNING > ERROR > CRITICAL
                      (Default level is INFO)
  -H, --host TEXT     Specify the host address for the Xinference server.
  -p, --port INTEGER  Specify the port number for the Xinference server.
  --help              Show this message and exit.

Commands:
  cached         List all cached models in Xinference.
  cal-model-mem  calculate gpu mem usage with specified model size and...
  chat           Chat with a running LLM.
  engine         Query the applicable inference engine by model name.
  generate       Generate text using a running LLM.
  launch         Launch a model with the Xinference framework with the...
  list           List all 

### Run Qwen-Chat

Xinference supports a variety of LLMs. Learn more in https://inference.readthedocs.io/en/latest/models/builtin/.

Let’s start by running a built-in model: `Qwen-1_8B-Chat`.


We can specify the model’s UID using the `--model-uid` or `-u` flag. If not specified, Xinference will generate it. This create a new model instance with unique ID `my-llvm`:


In [15]:
!xinference launch -u my-llm -n qwen-chat -s 1_8 -f pytorch -en transformers

Launch model name: qwen-chat with kwargs: {}
Traceback (most recent call last):
  File "/usr/local/bin/xinference", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/decorators.py", line 33, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xinference/dep

When you start a model for the first time, Xinference will download the model parameters from HuggingFace, which might take a few minutes depending on the size of the model weights. We cache the model files locally, so there’s no need to redownload them for subsequent starts.


## Interact with the running model

Congrats! You now have the model running by Xinference. Once the model is running, we can try it out either command line, via cURL, or via Xinference’s python client:



### 1.Use the OpenAI compatible endpoint

Xinference provides OpenAI-compatible APIs for its supported models, so you can use Xinference as a local drop-in replacement for OpenAI APIs. For example:


In [16]:
import openai

messages=[
    {
        "role": "user",
        "content": "Who are you?"
    }
]

client = openai.Client(api_key="empty", base_url=f"http://0.0.0.0:9997/v1")
client.chat.completions.create(
    model="my-llm",
    messages=messages,
)

InternalServerError: Error code: 500 - {'detail': '[address=127.0.0.1:38269, pid=3205] Trying to create tensor with negative dimension -6: [-6]'}

### 2. Send request using curl




In [11]:
!curl -k -X 'POST' -N \
  'http://127.0.0.1:9997/v1/chat/completions' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{ "model": "my-llm", "messages": [ {"role": "system", "content": "You are a helpful assistant." }, {"role": "user", "content": "What is the largest animal?"} ]}'

{"detail":"[address=127.0.0.1:38269, pid=3205] Trying to create tensor with negative dimension -8: [-8]"}

### 3. Use Xinference's Python client

In [12]:
from xinference.client import RESTfulClient
client = RESTfulClient("http://127.0.0.1:9997")
model = client.get_model("my-llm")
model.chat(
    prompt="hello",
    chat_history=[
    {
        "role": "user",
        "content": "What is the largest animal?"
    }]
)

TypeError: RESTfulChatModelHandle.chat() got an unexpected keyword argument 'prompt'

### 4. Langchain intergration

In [13]:
from langchain.llms import Xinference
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = Xinference(server_url='http://127.0.0.1:9997', model_uid='my-llm')

template = 'What is the largest {kind} on the earth?'

prompt = PromptTemplate(template=template, input_variables=['kind'])

llm_chain = LLMChain(prompt=prompt, llm=llm)

generated = llm_chain.run(kind='plant')
print(generated)

ModuleNotFoundError: No module named 'langchain_community'